# <h1>PREPROCESSING</h1>

## <h3>UPLOAD DATA</h3>

In [1]:
import pandas as pd
from binance.client import Client
from datetime import datetime, timedelta

api_key = 'Il7AIrG7k1CaXawPKVMSQAaiWEcYpIOswKzZIuObVPN3oO8JQEatLBHbkSzNk4ER'
secret_key = 'cEhGHkIuzgpoWmoiZuYrp8UEdNLb4Oh01pshY0xWOD3yOi0lruycCbdDm4t0Ap4H'

client = Client(api_key, secret_key)

start_date = "1 Jan, 2020"
end_date = "1 Feb, 2024"

# Convierte las cadenas de fecha en objetos datetime
start_date = datetime.strptime(start_date, '%d %b, %Y')
end_date = datetime.strptime(end_date, '%d %b, %Y')

# Obtener datos históricos de velas (candlesticks)
candlesticks = []
new_candles = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_15MINUTE, start_date.strftime('%d %b, %Y'), end_date.strftime('%d %b, %Y'), limit=1000)
candlesticks += new_candles



In [2]:
# Crear un DataFrame de Pandas
columns = ['open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'close_time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
df = pd.DataFrame(candlesticks, columns=columns)

In [3]:
# Convertir fechas de milisegundos a formato legible
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')

# Asegúrate de que los datos estén ordenados por fecha
df.sort_values('open_time', inplace=True)

# Crear un índice temporal único (time_idx)
df['time_idx'] = range(len(df))

df['series_id'] = 'default_series'

In [4]:
import talib
import pandas as pd

# Asumiendo que df es tu DataFrame y ya tiene las columnas 'Open', 'High', 'Low', 'Close', y 'Volume'

# Calcula el Simple Moving Average (SMA) de los precios de cierre
df['SMA_10'] = talib.SMA(df['Close'], timeperiod=15)  # SMA de 10 periodos

# 1. EMA de periodos cortos:
df['EMA_3'] = talib.EMA(df['Close'], timeperiod=3)
df['EMA_5'] = talib.EMA(df['Close'], timeperiod=5)

# Calcula el Relative Strength Index (RSI)
df['RSI'] = talib.RSI(df['Close'], timeperiod=15)

# Calcula el MACD
df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df['Close'], fastperiod=7, slowperiod=15, signalperiod=5)

# Calcula el Average True Range (ATR)
df['ATR'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=15)

# Calcula el On Balance Volume (OBV)
df['OBV'] = talib.OBV(df['Close'], df['Volume'])

# Patrones de velas japonesas
df['DOJI'] = talib.CDLDOJI(df['Open'], df['High'], df['Low'], df['Close'])
df['HAMMER'] = talib.CDLHAMMER(df['Open'], df['High'], df['Low'], df['Close'])
df['INV_HAMMER'] = talib.CDLINVERTEDHAMMER(df['Open'], df['High'], df['Low'], df['Close'])
df['SHOOTING_STAR'] = talib.CDLSHOOTINGSTAR(df['Open'], df['High'], df['Low'], df['Close'])
df['HANG_MAN'] = talib.CDLHANGINGMAN(df['Open'], df['High'], df['Low'], df['Close'])
df['ENGULFING'] = talib.CDLENGULFING(df['Open'], df['High'], df['Low'], df['Close'])

## <h3>VISUALIZATION</h3> 

In [5]:
df.shape

(143080, 29)

In [42]:
df.tail(71500)

,open_time,Open,High,Low,Close,Volume,close_time,Quote asset volume,Number of trades,Taker buy base asset volume,...,MACD_hist,ATR,OBV,DOJI,HAMMER,INV_HAMMER,SHOOTING_STAR,HANG_MAN,ENGULFING,price_movement
71580,2022-01-17 04:00:00,42647.398438,42736.160156,42601.000000,42728.09,331.919495,2022-01-17 04:14:59.999,14160266.37020220,11196,161.94207000,...,-10.384049,125.124725,150081.453125,0.0,0.0,0.0,0.0,0.0,0.0,2
71581,2022-01-17 04:15:00,42728.089844,42807.949219,42689.500000,42780.93,307.491974,2022-01-17 04:29:59.999,13147678.87797910,11189,174.37326000,...,-0.675803,124.679741,150388.953125,0.0,0.0,0.0,0.0,0.0,0.0,0
71582,2022-01-17 04:30:00,42780.941406,42784.949219,42628.011719,42684.66,372.489441,2022-01-17 04:44:59.999,15905950.29573330,13327,224.84339000,...,-3.077244,126.830429,150016.468750,0.0,0.0,0.0,0.0,0.0,-100.0,0
71583,2022-01-17 04:45:00,42684.648438,42747.921875,42615.550781,42622.99,226.102234,2022-01-17 04:59:59.999,9651887.15643650,8530,112.18195000,...,-7.536091,127.199730,149790.359375,0.0,0.0,100.0,0.0,0.0,0.0,0
71584,2022-01-17 05:00:00,42622.988281,42625.488281,42300.000000,42401.27,798.999939,2022-01-17 05:14:59.999,33911781.92508840,22971,339.82083000,...,-25.550250,140.419083,148991.359375,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143075,2024-01-31 23:00:00,42656.070312,42681.281250,42574.398438,42588.39,699.586182,2024-01-31 23:14:59.999,29824597.21813650,11356,319.77722000,...,33.810830,198.381256,462254.062500,0.0,0.0,0.0,0.0,0.0,0.0,2
143076,2024-01-31 23:15:00,42588.378906,42649.671875,42555.460938,42637.87,224.839035,2024-01-31 23:29:59.999,9581903.07889760,7533,126.49664000,...,35.267526,191.436508,462478.906250,0.0,0.0,0.0,0.0,0.0,0.0,0
143077,2024-01-31 23:30:00,42637.871094,42688.878906,42605.230469,42619.99,180.980804,2024-01-31 23:44:59.999,7715783.08681610,7621,102.51531000,...,32.535301,184.250732,462297.937500,0.0,0.0,0.0,0.0,0.0,0.0,0
143078,2024-01-31 23:45:00,42620.000000,42642.218750,42580.000000,42580.00,152.521835,2024-01-31 23:59:59.999,6499700.17232010,6137,67.36985000,...,26.159001,176.115356,462145.406250,0.0,0.0,0.0,0.0,0.0,0.0,2


## <h3>CLEAN DATA</h3>

In [39]:
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

In [40]:
columns_to_convert = ["Open", "High", "Low", "Volume", "SMA_10", "EMA_3", "EMA_5", 
                      "RSI", "MACD", "ATR", "OBV", "DOJI", "HAMMER", "INV_HAMMER", 
                      "SHOOTING_STAR", "HANG_MAN", "ENGULFING"]

# Convertir cada columna especificada a float32
for column in columns_to_convert:
    df[column] = df[column].astype('float32')

In [9]:
import pandas as pd

# Add a new column with the shifted close prices to compare with
df['close_shifted'] = df['Close'].shift(-1)

# Calculate the encoded 'price_movement' based on your conditions
df['price_movement'] = 1  # Initialize as no change
df.loc[df['Close'] < df['close_shifted'], 'price_movement'] = 2  # Price goes up
df.loc[df['Close'] > df['close_shifted'], 'price_movement'] = 0  # Price goes down

# Drop the 'close_shifted' as it was only used for the calculation
df.drop('close_shifted', axis=1, inplace=True)

In [10]:
# Eliminar filas donde todos los valores son NaN
df.dropna(inplace=True)

In [11]:
import pandas as pd
import numpy as np

# Assuming your DataFrame is named 'df'

# 1. Calculate the number of rows to remove:
current_rows = df.shape[0]
remainder = current_rows % 100
rows_to_remove = remainder

# 2. Perform random undersampling:
if rows_to_remove > 0:
    df = df.tail(current_rows - rows_to_remove)  # Set random_state for reproducibility

# Now your df.shape[0] should be a multiple of 100 
print(df.shape)

(143000, 30)


In [41]:
df.dtypes

open_time                       datetime64[ns]
Open                                   float32
High                                   float32
Low                                    float32
Close                                  float64
Volume                                 float32
close_time                      datetime64[ns]
Quote asset volume                      object
Number of trades                         int64
Taker buy base asset volume             object
Taker buy quote asset volume            object
Ignore                                  object
time_idx                                 int64
series_id                               object
SMA_10                                 float32
EMA_3                                  float32
EMA_5                                  float32
RSI                                    float32
MACD                                   float32
MACD_signal                            float64
MACD_hist                              float64
ATR          

# <h1>MODEL</h1>

## <h3>IMPLEMENTATION</h3>

In [ ]:
# Crear un texto que describe la secuencia de movimientos
movements_text = " ".join(f"En el paso {i}, el movimiento fue {movement}." 
                          for i, movement in enumerate(df['price_movement']))

print(movements_text[:500])  # Mostrar los primeros 500 caracteres para revisar

In [82]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Cargar modelo y tokenizador
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Tokenizar el texto de entrada
inputs = tokenizer(movements_text, return_tensors='pt', max_length=512, truncation=True)

# Generar una secuencia continuada a partir de la entrada
output_sequences = model.generate(
    input_ids=inputs['input_ids'],
    max_length=600,  # Extender a una longitud mayor para ver la continuación generada
    temperature=1.0,  # Ajustar para variar la creatividad de las predicciones
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.2,
)

# Decodificar y mostrar el texto generado
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(generated_text)

In [90]:
train_dataset = PriceMovementDataset('train', price_movement[:9980])
test_dataset = PriceMovementDataset('test', price_movement[9980:10000])

In [91]:
# create a GPT instance
from mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = train_dataset.get_vocab_size()
model_config.block_size = train_dataset.get_block_size()
model = GPT(model_config)

In [88]:
# create a Trainer object
from mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-4 # the model we're using is so small that we can go a bit faster
train_config.max_iters = 6000
train_config.num_workers = 0
trainer = Trainer(train_config, model, train_dataset)

In [56]:
def batch_end_callback(trainer):
    if trainer.iter_num % 100 == 0:
        print(f"iter_dt {trainer.iter_dt * 1000:.2f}ms; iter {trainer.iter_num}: train loss {trainer.loss.item():.5f}")
trainer.set_callback('on_batch_end', batch_end_callback)

trainer.run()

TypeError: Trainer.__init__() got an unexpected keyword argument 'auto_lr_find'

In [92]:
# now let's perform some evaluation
model.eval();

Epoch 1/100 Loss: 0.7015154431892346
Epoch 2/100 Loss: 0.6972861182086075
Epoch 3/100 Loss: 0.6970525249650207
Epoch 4/100 Loss: 0.6968987340414071
Epoch 5/100 Loss: 0.6968358620812621
Epoch 6/100 Loss: 0.6968169391155243


KeyboardInterrupt: 

: 

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.utils import set_seed
set_seed(3407)

def eval_split(trainer, split, max_batches):
    dataset = {'train':train_dataset, 'test':test_dataset}[split]
    n = train_dataset.length # naugy direct access shrug
    results = []
    mistakes_printed_already = 0
    loader = DataLoader(dataset, batch_size=100, num_workers=0, drop_last=False)
    for b, (x, y) in enumerate(loader):
        x = x.to(trainer.device)
        y = y.to(trainer.device)
        # isolate the input pattern alone
        inp = x[:, :n]
        sol = y[:, -n:]
        # let the model sample the rest of the sequence
        cat = model.generate(inp, n, do_sample=False) # using greedy argmax, not sampling
        sol_candidate = cat[:, n:] # isolate the filled in sequence
        # compare the predicted sequence to the true sequence
        #print("sol: ", sol[10:15,-8:])
        #print("sol_candidate: ", sol_candidate[10:15,-8:])
        correct = (sol == sol_candidate).all(1).cpu() # Software 1.0 vs. Software 2.0 fight RIGHT on this line haha
        #print("correct:",correct)
        for i in range(x.size(0)):
            results.append(int(correct[i]))
            if not correct[i] and mistakes_printed_already < 3: # only print up to 5 mistakes to get a sense
                mistakes_printed_already += 1
                print("GPT claims that with %s the future price dynamics is %s but gt is %s" % (inp[i].tolist(), sol_candidate[i].tolist(), sol[i].tolist()))
        if max_batches is not None and b+1 >= max_batches:
            break
    rt = torch.tensor(results, dtype=torch.float)
    print("%s final score: %d/%d = %.2f%% correct" % (split, rt.sum(), len(results), 100*rt.mean()))
    return rt.sum()

# run a lot of examples from both train and test through the model and verify the output correctness
with torch.no_grad():
    train_score = eval_split(trainer, 'train', max_batches=50)
    test_score  = eval_split(trainer, 'test',  max_batches=50)

# Evaluation


In [80]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [81]:
from sklearn.metrics import classification_report, confusion_matrix

model.eval()  # Poner el modelo en modo de evaluación

true_labels = []
predictions = []

with torch.no_grad():  # Indicar a PyTorch que no calcule gradientes
    for X_batch, y_batch in test_loader:
        y_pred = model(X_batch)
        _, predicted = torch.max(y_pred, 1)
        
        true_labels.extend(y_batch.numpy())
        predictions.extend(predicted.numpy())

# Calcular métricas de evaluación
print(classification_report(true_labels, predictions))
print(confusion_matrix(true_labels, predictions))

              precision    recall  f1-score   support

           0       0.48      0.25      0.33     11698
           1       0.00      0.00      0.00        15
           2       0.51      0.75      0.61     12311

    accuracy                           0.50     24024
   macro avg       0.33      0.33      0.31     24024
weighted avg       0.50      0.50      0.47     24024

[[2871    0 8827]
 [   5    0   10]
 [3085    0 9226]]


d:\Programas\Anaconda\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Programas\Anaconda\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Programas\Anaconda\envs\myenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
